In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.io as io
import matplotlib.pyplot as plt

In [ ]:
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# Classifiers

from google.colab import drive                  # Импортируем необходимые библиотеки
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!ls -l '/content/gdrive/'                  # Монтируем диск Google Drive

total 4
drwx------ 12 root root 4096 Jun  5 10:35 MyDrive


In [ ]:
# Проверяем содержимое корневой директории Google Drive
spam_train = io.loadmat("/content/gdrive/MyDrive/Colab Notebooks/ML/spamTrain.mat")
spam_test = io.loadmat("/content/gdrive/MyDrive/Colab Notebooks/ML/spamTest.mat")

In [ ]:
# Загружаем данные из файлов spamTrain.mat и spamTest.mat
spam_X_train, spam_y_train = spam_train['X'], spam_train['y']
spam_X_test, spam_y_test = spam_test['Xtest'], spam_test['ytest']


In [ ]:
# Извлекаем обучающие и тестовые признаки и метки классов
print(np.unique(spam_X_train))
print(spam_X_train.shape,spam_y_train.shape)

[0 1]
(4000, 1899) (4000, 1)


In [ ]:
# Выводим уникальные значения в обучающих признаках и размерности обучающих данных

def calc_metrics_cls(true, pred):
    stats = {
        'Accuracy': round(metrics.accuracy_score(true, pred), 2),
        'ROC AUC': round(metrics.roc_auc_score(true, pred), 2),
        'F1': round(metrics.f1_score(true, pred), 2)
    }
    print(f"Accuracy: {stats['Accuracy']}\nROC AUC: {stats['ROC AUC']}\nF1: {stats['F1']}")
    return stats

In [ ]:
# Определяем функцию для вычисления метрик качества классификации
cls_models_stats = {}

DecisionTree

In [ ]:
# Создаем пустой словарь для хранения статистик классификаторов

cls_model_dt = DecisionTreeClassifier()
cls_model_dt.fit(spam_X_train, spam_y_train)
y_pred_cls_dt = cls_model_dt.predict(spam_X_test)
cls_models_stats['DecisionTree'] = calc_metrics_cls(spam_y_test, y_pred_cls_dt)

Accuracy: 0.93
ROC AUC: 0.93
F1: 0.89


In [ ]:
# Обучаем модель дерева принятия решений на обучающих данных, выполняем прогноз и вычисляем метрики качества
cls_model_svm = SVC()
cls_model_svm.fit(spam_X_train, spam_y_train)
y_pred_cls_dt = cls_model_svm.predict(spam_X_test)
cls_models_stats['SVM'] = calc_metrics_cls(spam_y_test, y_pred_cls_dt)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.99
ROC AUC: 0.98
F1: 0.98


In [ ]:
# Обучаем модель метода опорных векторов на обучающих данных, выполняем прогноз и вычисляем метрики качества
# Добавьте другие классификаторы по желанию

# Реализация нейронной сети
cls_model_nn = Sequential()
cls_model_nn.add(Dense(64, input_dim=spam_X_train.shape[1], activation='relu'))
cls_model_nn.add(Dense(32, activation='relu'))
cls_model_nn.add(Dense(1, activation='sigmoid'))

In [ ]:
cls_model_nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
cls_model_nn.fit(spam_X_train, spam_y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
125/125 [==============================] - 1s 5ms/step - loss: 0.1975 - accuracy: 0.9358
Epoch 2/10
125/125 [==============================] - 1s 7ms/step - loss: 0.0557 - accuracy: 0.9895
Epoch 3/10
125/125 [==============================] - 1s 7ms/step - loss: 0.0163 - accuracy: 0.9965
Epoch 4/10
125/125 [==============================] - 1s 7ms/step - loss: 0.0072 - accuracy: 0.9995
Epoch 5/10
125/125 [==============================] - 1s 6ms/step - loss: 0.0041 - accuracy: 0.9998
Epoch 6/10
125/125 [==============================] - 1s 6ms/step - loss: 0.0027 - accuracy: 0.9998
Epoch 7/10
125/125 [==============================] - 1s 5ms/step - loss: 0.0020 - accuracy: 0.9998
Epoch 8/10
125/125 [==============================] - 1s 4ms/step - loss: 0.0015 - accuracy: 0.9998
Epoch 9/10
125/125 [==============================] - 1s 4ms/step - loss: 0.0012 - accuracy: 0.9998
Epoch 10/10
125/125 [==============================] - 1s 4ms/step - loss: 0.0011 - accuracy: 0.9998

In [ ]:
y_pred_cls_nn = (cls_model_nn.predict(spam_X_test) > 0.5).astype(int)

y_pred_cls_nn = y_pred_cls_nn.flatten()

32/32 [==============================] - 1s 6ms/step


In [ ]:
cls_models_stats['NeuralNetwork'] = calc_metrics_cls(spam_y_test, y_pred_cls_nn)

Accuracy: 0.99
ROC AUC: 0.99
F1: 0.99


In [ ]:
model_cls_compare = pd.DataFrame(cls_models_stats).T
model_cls_compare

# Создаем таблицу для сравнения результатов метрик классификации разных моделей

,Accuracy,ROC AUC,F1
DecisionTree,0.93,0.93,0.89
SVM,0.99,0.98,0.98
NeuralNetwork,0.99,0.99,0.99
